# install & load modules

In [ ]:
### install module
#!pip install nibabel
#!pip3 install numpy
#!pip3 install ipdb
#!pip3 install nilearn
#!pip3 install scikit-image
#!pip install matplotlib

### do not need to install for this code
#!pip3 install torch
#!pip3 install monai

In [ ]:
### load module
import nibabel as nib
import numpy as np
import ipdb
from ipdb import set_trace

import re #regular expression
import os
import csv
from nilearn import plotting
from skimage.transform import resize
#import torch
#from monai.transforms import ScaleIntensity

# main code

In [ ]:
### try to environment setting
#os.environ['FREESURFER_HOME']='usr/local/freesurfer/7.2.0'

#subprocess.call('/bin/bash -c "$GREPDB"', shell=True, env={'GREPDB':'$FREESURFER_HOME/SetUpFreeSurfer.sh'})
#os.environ['FREESURFER_HOME']
#!/bin/bash -c 'source /usr/local/freesurfer/7.2.0/SetUpFreeSurfer.sh'
#cd /usr/local/freesurfer/7.2.0

In [ ]:
arr = range(0, 64, 64 // 15)
ratio_map = {}
count = [0]
padding = 10

# main function
def dive(root): 
    for f in os.listdir(root):
        path = f'{root}/{f}'
        
        # ends with ".mgz" or ".nii"
        if f.endswith('T1.mgz'):
            # (1) mri_convert
            nii_path=mriconvert(root, f) 
            
            break
            
            # (2) skull-stripping
            f_brain = bet(nii_path,root,f)
            
            # (3) cut unnecessary part
            clean_space(root, f_brain) #load clean space function
            
            # (4) show image
            path = f'{root}/{f_brain}'
            img = nib.load(path)
            np_img = np.array(img.get_fdata())
            data = resize(np_img, (64, 64, 64), mode='constant')
            
            #Min-max scaling
            #transform = ScaleIntensity(minv=-1.0, maxv=1.0) #[-1,1] MinMax scaler
            #data = transform(data)
            
            data_nii = nib.Nifti1Image(data, np.eye(4))
            
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
            plotting.show()
            

            
            count[0] += 1
            print(f'count: {count[0]} processed: {root}/{f}')
        
        # ends with ".npy"
        if f.endswith('.npy'):
            
            #cut unnecessary part
            clean_space(root, f) #load clean space function
            
            # show image (cz .npy is already np array -> you don't need to run np.array())
            np_img = np.load(path)
            data = resize(np_img, (64, 64, 64), mode='constant') #resize the image

            #Min-max scaling
            # scaling code does not work -> exclude min-max scaling
            #transform = ScaleIntensity(minv=-1.0, maxv=1.0) #[-1,1] MinMax scaler
            #data = transform(data)

            data_nii = nib.Nifti1Image(data, np.eye(4))
            
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='x')
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='y')
            plotting.plot_img(data_nii,cut_coords=arr,draw_cross=False,annotate=False,black_bg=True,display_mode='z')
            plotting.show()
            

            count[0] += 1
            print(f'count: {count[0]} processed: {root}/{f}')
            
        # [file tree] directory - file  > recursively work
        if f.endswith('flair.nii.gz') or f.endswith('t2.nii.gz'):
            os.system(f'gunzip {path}')
        elif os.path.isdir(path):
            dive(path)

# mgz2nii - freesurfer is needd to run this code          
def mriconvert(root, f):
    print(1)
    subjectname=re.sub('.T1.mgz','',f)
    mgz_path = rf'{root}/{f}'
    nii_path = rf'{root}/{subjectname}.T1.nii'
    os.system(f'mri_convert {mgz_path} {nii_path}') #mri_convert command
    print(2)
    return nii_path

# skull-tripping
#bet $ADNI_T1/${SUBJECT}/T1.anat/T1_to_MNI_nonlin.nii.gz $ADNI_T1/${SUBJECT}/T1.anat/brain_to_MNI_nonlin.nii.gz
def bet(nii_path,root,f):
    subjectname=re.sub('.T1.mgz','',f)
    T1_path = nii_path
    brain_path = rf'{root}/{subjectname}.brain.nii'
    os.system(f'bet -m {T1_path} {brain_path}') #bet command
    return f'{subjectname}.brain.nii'

### clean function
def clean_axis(np_img, path, axis=0):
    x_len, y_len, z_len = len(np_img), len(np_img[0]), len(np_img[0, 0])
    zero_mask = np.zeros(0)
    del_inds = []
    save_i = -1
    iter_len = 0
    continuity_warning = 0
    
    if axis == 0: 
        zero_mask = np.zeros((y_len, z_len))
        iter_len = x_len
    elif axis == 1: 
        zero_mask = np.zeros((x_len, z_len))
        iter_len = y_len
    elif axis == 2: 
        zero_mask = np.zeros((x_len, y_len))
        iter_len = z_len
    for i in range(iter_len):
        plane = np.sum(np_img[i] != zero_mask) if axis == 0 else np.sum(np_img[:, i] != zero_mask) if axis == 1 else np.sum(np_img[:, :, i] != zero_mask) 
        if plane == 0: 
            del_inds.append(i)
            if save_i + 1 != i: continuity_warning += 1
            save_i = i

    # cannot allow two breakpoints in del_inds
    if continuity_warning >= 2: print(f'\x1b[31m broken del_inds: {path} \n axis: {axis} \x1b[0m') 
        
    # remove empty slices (every other slice to maintain the empty space ratio)
    np_img = np.delete(np.swapaxes(np_img, 0, axis), del_inds, 0)
    # add padding
    zero_mask = np.zeros((len(np_img[0]), len(np_img[0, 0])))
    np_img = np.insert(np_img, [0 for i in range(padding)], zero_mask, 0)
    np_img = np.insert(np_img, [len(np_img) for i in range(padding)], zero_mask, 0)
    
    return np.swapaxes(np_img, 0, axis)


# clean the 0-intensity space surrounding the brain
def clean_space(root, f):
    img_pth = rf'{root}/{f}'
    subjectname=re.sub('.brain.nii','',f)
    
    
    np_img = np.load(img_pth)
    
    #img = nib.load(img_pth)
    #np_img = np.array(img.get_fdata())
        
    #clean axes (cleaned img)
    np_img = clean_axis(np_img, img_pth, axis=0)
    np_img = clean_axis(np_img, img_pth, axis=1)
    np_img = clean_axis(np_img, img_pth, axis=2)
    
    # make .mgz / .nii with cleaned img
    if f.endswith('mgz'):
        # write to disk
        feat = nib.freesurfer.mghformat.MGHImage(np_img, img.affine, img.header)
    elif f.endswith('nii'):
        img_pth_nii = rf'{root}/{subjectname}.brain_crop.nii'
        img_pth_npy = rf'{root}/{subjectname}.brain_crop.npy'
        feat = nib.Nifti1Image(np_img, img.affine, header=img.header) #chage data as nifti format
        nib.save(feat, img_pth_nii)
        
        np.save(img_pth_npy, np_img) #

# visualization w/ 1 toy subject

In [ ]:
!ls

In [ ]:
# before cutting
if __name__ == '__main__':
    dive('/home/connectome/bettybetty3k/0.myResearch/3DCNN')

In [ ]:
import subprocess
#subprocess.call('/bin/bash -c "$GREPDB"', shell=True, env={'GREPDB':'mri_convert'})
sp = subprocess.Popen(["/bin/bash","-i","-c","mri_convert"])
sp.communicate()

In [ ]:
# after cutting
if __name__ == '__main__':
    dive('/home/connectome/bettybetty3k/0.myResearch/3DCNN')

# visualization w/ all subjects

In [ ]:
if __name__ == '__main__':
    dive('/scratch/bigdata/ABCD/freesurfer/smri/freesurfer_smri') #storage-data / others-scratch

In [ ]:
# len('/scratch/bigdata/ABCD/freesurfer/smri/freesurfer_smri/sub-NDARINVZZZP87KR') #73

In [ ]:
# cd /scratch/bigdata/ABCD/freesurfer/smri/freesurfer_smri